In [1]:
%cd ~/Workspace/kaggle-2019Q3-siim/


/home/chengjiun/Workspace/kaggle-2019Q3-siim


# top

- copied from https://www.kaggle.com/chengjiun/siim-data-repack-and-image-statistics/edit
- to rescale the image to smaller size, and make mask for fastai

In [2]:
import os
import cv2
from glob import glob
import pydicom
from tqdm import tqdm_notebook as tqdm
import zipfile
import io
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage import exposure
from TOOLS.mask_functions import *

In [3]:
sz = 512
sz0 = 1024
PATH_TRAIN = 'input/dicom-images-train/'
PATH_TEST = 'input/dicom-images-test/'
train_out = f'input/train_{sz}.zip'
test_out = f'input/test_{sz}.zip'
mask_out = f'input/masks_{sz}.zip'
train = glob(os.path.join(PATH_TRAIN, '*/*/*.dcm'))
test = glob(os.path.join(PATH_TEST, '*/*/*.dcm'))

In [4]:
df = pd.read_csv('input/train-rle.csv').set_index('ImageId')
idxs = set(df.index)
train_names = []
for f in train: #remove images without labels
    name = f.split('/')[-1][:-4]
    if name in idxs: train_names.append(f)

In [5]:
def convert_images(filename, arch_out, sz=sz):
    ds = pydicom.read_file(str(filename))
    img = ds.pixel_array
    img = cv2.resize(img, (sz, sz))
    img = exposure.equalize_adapthist(img) # contrast correction
    x_tot = img.mean() #image statistics
    x2_tot = (img**2).mean()
    img = ((img*255)).clip(0,255).astype(np.uint8)
    output = cv2.imencode('.png',img)[1]
    name = filename.split('/')[-1][:-4] + '.png'
    arch_out.writestr(name, output)
    return x_tot, x2_tot

def get_stats(stats): # get dataset statistics 
    x_tot, x2_tot = 0.0, 0.0
    for x, x2 in stats:
        x_tot += x
        x2_tot += x2
    
    img_avr =  x_tot/len(stats)
    img_std =  np.sqrt(x2_tot/len(stats) - img_avr**2)
    print('mean:',img_avr, ', std:', img_std)

# write images

In [6]:

trn_stats = []
with zipfile.ZipFile(train_out, 'w') as arch:
    for fname in tqdm(train_names, total=len(train_names)):
        trn_stats.append(convert_images(fname,arch))

test_stats = []        
with zipfile.ZipFile(test_out, 'w') as arch:
    for fname in tqdm(test, total=len(test)):
        test_stats.append(convert_images(fname,arch))

In [7]:
get_stats(trn_stats)
get_stats(test_stats)

mean: 0.5292001691897469 , std: 0.25880564368364406
mean: 0.5265718130841706 , std: 0.2589975116265946


In [14]:
get_stats(trn_stats)
get_stats(test_stats)

mean: 0.5403185386035164 , std: 0.2643158961302565
mean: 0.5380199093045763 , std: 0.264917379950516


# write mask

In [11]:
mask_coverage = []
mask_count = 0
with zipfile.ZipFile(mask_out, 'w') as arch:
    for idx in tqdm(idxs):
        masks = df.loc[idx,' EncodedPixels']
        img = np.zeros((sz0,sz0))
        #do conversion if mask is not " -1"
        if(type(masks) != str or (type(masks) == str and masks != ' -1')):
            if(type(masks) == str): masks = [masks]
            else: masks = masks.tolist()
            mask_count +=1
            for mask in masks:
                img += rle2mask(mask, sz0, sz0).T
        mask_coverage.append(img.mean())
        img = cv2.resize(img, (sz, sz))
        output = cv2.imencode('.png',img)[1]
        name = idx + '.png'
        arch.writestr(name, output)

print('mask coverage:', np.mean(mask_coverage)/255, ', mask count:', mask_count)


mask coverage: 0.0031809534792040213 , mask count: 2379


# show one example

In [10]:
idx = 736
with zipfile.ZipFile(train_out, 'r') as arch:
    fname = sorted(arch.namelist())[idx]
    flags = cv2.IMREAD_GRAYSCALE
    img = cv2.imdecode(np.frombuffer(arch.read(fname), np.uint8), flags)
    
with zipfile.ZipFile(mask_out, 'r') as arch:
    fname = sorted(arch.namelist())[idx]
    flags = cv2.IMREAD_GRAYSCALE
    mask = cv2.imdecode(np.frombuffer(arch.read(fname), np.uint8), flags)
    
plt.figure()
plt.imshow(Image.fromarray(img))
plt.imshow(Image.fromarray(mask), alpha=0.2)
plt.show()

IndexError: list index out of range